In [1]:
# Demora 21min e 33s para rodar 1 mês dos dados de 5min
import time

# Todas as variáveis para as Américas
- Para processar 2 dias de arquivos de 5 min: Wall time: 4min 27s
- 1.3 MB

In [2]:
%%time
#==================================================================================================================#
#                                              IMPORTA BIBLIOTECAS
#==================================================================================================================#
import os
import pandas as pd
from datetime import datetime
import glob
import xarray as xr
import time
import numpy as np

#==================================================================================================================#
#                                               DEFINE CAMINHO
#==================================================================================================================#
dir_5min = 'output/5min/'
dir_diario = 'output/diario/'

#==================================================================================================================#
#                                            DEFINE O PERÍODO DE PROCESSAMENTO
#==================================================================================================================#
# ano, mes e dia INICIAL do período
anoi = 2018
mesi = 1
diai = 1

# ano, mes e dia FINAL do período
anof = 2018
mesf = 1
diaf = 2

# data desejada
date_in = datetime(anoi, mesi, diai)
date_ini = date_in.strftime('%Y%m%d')

date_en = datetime(anof, mesf, diaf)
date_end = date_en.strftime('%Y%m%d')

#==================================================================================================================#
#                                               LOOP NOS DIAS
#==================================================================================================================#
for file in pd.date_range(date_ini, date_end, freq='d'):

    #---------------------------------------------#
    #             Extrai a data
    #---------------------------------------------#
    ano = file.strftime('%Y')
    mes = file.strftime('%m')
    dia = file.strftime('%d')
    print(f'######################## {ano}-{mes}-{dia} ########################')

    #---------------------------------------------#
    #           Leitura dos dados
    #---------------------------------------------#
    # caminho do diretório dos arquivos
    dir_files = f'{dir_5min}{ano}/{mes}/' 

    # lista os arquivos do dia
    files = sorted(glob.glob(f'{dir_files}S11635949_{ano}{mes}{dia}*nc')) # S11635949_201901140445.nc
    print('1. Montou a lista dos arquivos=', len(files))

    # verifica se existe arquivos daquele dia
    if len(files) > 0:

        # leitura dos dados diários
        glm_dia = xr.open_mfdataset(files, concat_dim='time', combine='nested', decode_times=False) #.sel(lon=slice(lonmin,lonmax), lat=slice(latmin, latmax))
        print('2. Leu os arquivos daquele dia')

        #---------------------------------------------#
        #           Salva arquivo Netcdf
        #---------------------------------------------#
        # Tutorial sobre como criar DataSet com a biblioteca Xarray: https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html
        # variáveis
        
        # eventos, grupos e flashes-GASTA MUIOT TEMPO
        event_dia = np.array(glm_dia['event'].sum(dim='time'))
        group_dia = np.array(glm_dia['group'].sum(dim='time'))
        flash_dia = np.array(glm_dia['flash'].sum(dim='time'))

        # energia e duração
        flash_energy_dia = np.array(glm_dia['flash_energy'].mean(dim='time'))
        duration_flash_dia = np.array(glm_dia['duration_flash'].mean(dim='time'))

        # fecha o arquivo lido
        glm_dia.close()        
        print('3. Calculou o valor total/média')

        # adiciona a dimensão do tempo
        event_dia = event_dia[np.newaxis, :, :]
        group_dia = group_dia[np.newaxis, :, :]
        flash_dia = flash_dia[np.newaxis, :, :]
        
        flash_energy_dia = flash_energy_dia[np.newaxis, :, :]
        duration_flash_dia = duration_flash_dia[np.newaxis, :, :]
        print('4. Criou a dimensão do tempo')

        # latitudes, longitudes e tempo
        lats = np.array(glm_dia['lat'])
        lons = np.array(glm_dia['lon'])
        times = pd.date_range(f'{ano}-{mes}-{dia}', periods=1)

        # define encoding para compactação dos dados
        encoding = {'event': {'zlib': True},
                    'group': {'zlib': True},
                    'flash': {'zlib': True},
                    'flash_energy': {'zlib': True},
                    'duration_flash': {'zlib': True}}

        # define variáveis
        data_vars = {'event':( ('time', 'lat', 'lon'), event_dia, {'units': 'ocorrências/64km2', 'long_name': 'Eventos', '_FillValue': 0} ),
                     'group':( ('time', 'lat', 'lon'), group_dia, {'units': 'ocorrências/64km2', 'long_name': 'Grupos', '_FillValue': 0} ),
                     'flash':( ('time', 'lat', 'lon'), flash_dia, {'units': 'ocorrências/64km2', 'long_name': 'Flashes', '_FillValue': 0} ),
                     'flash_energy':( ('time', 'lat', 'lon'), flash_energy_dia, {'units': 'Joule', 'long_name': 'Densidade de Energia Radiante do Flash', '_FillValue': 0} ),
                     'duration_flash':( ('time', 'lat', 'lon'), duration_flash_dia, {'units':'segundos', 'long_name': 'Duração Média do Flash Considerando o Primeiro e Último Evento', '_FillValue': 0} )}

        # define coordenadas
        coords = {'time': times, 'lat': lats, 'lon': lons}

        # define atributos
        attrs = {'description': 'Dados do Sensor GLM abordo do satélite GOES-16 Reamostrado para Escala Diária', 'creation_data': str(datetime.now()), 'author':'Enrique V. Mattos', 'email': 'enrique@unifei.edu.br'}
        print('5. Definou variáveis do DataSet')
        
        # gera Dataset
        glm_dia_netcdf = xr.Dataset(data_vars = data_vars, coords = coords, attrs = attrs)
        print('6. Gerou DataSet')

        # cria a pasta de saída
        os.makedirs(f'{dir_diario}{ano}/{mes}', exist_ok=True)

        # salva arquivo netcdf
        glm_dia_netcdf.to_netcdf(f'{dir_diario}{ano}/{mes}/GOES16_GLM_{ano}{mes}{dia}.nc', encoding=encoding)
        print('7. Salvou arquivo netcdf \n')

######################## 2018-01-01 ########################
Montou a lista dos arquivos= 288
Leu os arquivos daquele dia
Calculou o valor total/média
Criou a dimensão do tempo
Gerou DataSet
Salvou arquivo netcdf 

######################## 2018-01-02 ########################
Montou a lista dos arquivos= 288
Leu os arquivos daquele dia
Calculou o valor total/média
Criou a dimensão do tempo
Gerou DataSet
Salvou arquivo netcdf 

CPU times: total: 4min 38s
Wall time: 4min 27s


# Apenas Flash e para as Américas
- Para processar 2 dias de arquivos de 5 min: Wall time: 1min 10s
- 151 KB

In [ ]:
%%time
#==================================================================================================================#
#                                              IMPORTA BIBLIOTECAS
#==================================================================================================================#
import os
import pandas as pd
from datetime import datetime
import glob
import xarray as xr
import time
import numpy as np

#==================================================================================================================#
#                                               DEFINE CAMINHO
#==================================================================================================================#
dir_5min = 'output/5min/'
dir_diario = 'output/diario/'

#==================================================================================================================#
#                                            DEFINE O PERÍODO DE PROCESSAMENTO
#==================================================================================================================#
# ano, mes e dia INICIAL do período
anoi = 2018
mesi = 7
diai = 1

# ano, mes e dia FINAL do período
anof = 2018
mesf = 12
diaf = 31

# data desejada
date_in = datetime(anoi, mesi, diai)
date_ini = date_in.strftime('%Y%m%d')

date_en = datetime(anof, mesf, diaf)
date_end = date_en.strftime('%Y%m%d')

#==================================================================================================================#
#                                               LOOP NOS DIAS
#==================================================================================================================#
for file in pd.date_range(date_ini, date_end, freq='d'):

    #---------------------------------------------#
    #             Extrai a data
    #---------------------------------------------#
    ano = file.strftime('%Y')
    mes = file.strftime('%m')
    dia = file.strftime('%d')
       
    print('####################################################################')
    print('                PROCESSANDO ===>>>', f'{ano}-{mes}-{dia}')
    print('####################################################################')
    
    #---------------------------------------------#
    #           Leitura dos dados
    #---------------------------------------------#
    # caminho do diretório dos arquivos
    dir_files = f'{dir_5min}{ano}/{mes}/' 

    # lista os arquivos do dia
    files = sorted(glob.glob(f'{dir_files}S11635949_{ano}{mes}{dia}*nc')) # S11635949_201901140445.nc
    print('1. Montou a lista dos arquivos=', len(files))

    # verifica se existe arquivos daquele dia
    if len(files) > 0:

        # leitura dos dados diários
        glm_dia = xr.open_mfdataset(files, concat_dim='time', combine='nested', decode_times=False) #.sel(lon=slice(lonmin,lonmax), lat=slice(latmin, latmax))
        print('2. Leu os arquivos daquele dia')

        #---------------------------------------------#
        #           Salva arquivo Netcdf
        #---------------------------------------------#
        # Tutorial sobre como criar DataSet com a biblioteca Xarray: https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html
        # variáveis
        
        # flashes-GASTA MUIOT TEMPO
        flash_dia = np.array(glm_dia['flash'].sum(dim='time'))

        # fecha o arquivo lido
        glm_dia.close()        
        print('3. Calculou o valor total/média')

        # adiciona a dimensão do tempo
        flash_dia = flash_dia[np.newaxis, :, :]
        print('4. Criou a dimensão do tempo')

        # latitudes, longitudes e tempo
        lats = np.array(glm_dia['lat'])
        lons = np.array(glm_dia['lon'])
        times = pd.date_range(f'{ano}-{mes}-{dia}', periods=1)

        # define encoding para compactação dos dados
        encoding = {'flash': {'zlib': True}}

        # define variáveis
        data_vars = {'flash':( ('time', 'lat', 'lon'), flash_dia, {'units': 'ocorrências/64km2', 'long_name': 'Flashes', '_FillValue': 0} )}

        # define coordenadas
        coords = {'time': times, 'lat': lats, 'lon': lons}

        # define atributos
        attrs = {'description': 'Dados do Sensor GLM abordo do satélite GOES-16 Reamostrado para Escala Diária', 'creation_data': str(datetime.now()), 'author':'Enrique V. Mattos', 'email': 'enrique@unifei.edu.br'}
        print('5. Definou variáveis do DataSet')
        
        # gera Dataset
        glm_dia_netcdf = xr.Dataset(data_vars = data_vars, coords = coords, attrs = attrs)
        print('6. Gerou DataSet')

        # cria a pasta de saída
        os.makedirs(f'{dir_diario}{ano}/{mes}', exist_ok=True)

        # salva arquivo netcdf
        glm_dia_netcdf.to_netcdf(f'{dir_diario}{ano}/{mes}/GOES16_GLM_flashes_{ano}{mes}{dia}.nc', encoding=encoding)
        print('7. Salvou arquivo netcdf \n')

####################################################################
                PROCESSANDO ===>>> 2018-07-01
####################################################################
1. Montou a lista dos arquivos= 288
2. Leu os arquivos daquele dia
3. Calculou o valor total/média
4. Criou a dimensão do tempo
5. Definou variáveis do DataSet
6. Gerou DataSet
7. Salvou arquivo netcdf 

####################################################################
                PROCESSANDO ===>>> 2018-07-02
####################################################################
1. Montou a lista dos arquivos= 288
2. Leu os arquivos daquele dia
3. Calculou o valor total/média
4. Criou a dimensão do tempo
5. Definou variáveis do DataSet
6. Gerou DataSet
7. Salvou arquivo netcdf 

####################################################################
                PROCESSANDO ===>>> 2018-07-03
####################################################################
1. Montou a lista dos arquivos= 288
2. L

# Apenas Flash e apenas para o Brasil
- Para processar 2 dias de arquivos de 5 min: Wall time: 1min 7s. 1 Mês Wall time: 21min 11s
- 95 KB

In [5]:
glm_dia

<xarray.Dataset>
Dimensions:         (time: 288, lat: 1664, lon: 1664)
Coordinates:
  * time            (time) float64 1.768e+07 1.768e+07 ... 1.768e+07 1.768e+07
  * lat             (lat) float32 -66.56 -66.48 -66.4 ... 66.32 66.4 66.48
  * lon             (lon) float32 -141.6 -141.5 -141.4 ... -8.68 -8.6 -8.52
Data variables:
    spatial_ref     (time) int32 -2147483647 -2147483647 ... -2147483647
    event           (time, lat, lon) float32 dask.array<chunksize=(1, 1664, 1664), meta=np.ndarray>
    flash           (time, lat, lon) float32 dask.array<chunksize=(1, 1664, 1664), meta=np.ndarray>
    group           (time, lat, lon) float32 dask.array<chunksize=(1, 1664, 1664), meta=np.ndarray>
    event_energy    (time, lat, lon) float32 dask.array<chunksize=(1, 1664, 1664), meta=np.ndarray>
    flash_energy    (time, lat, lon) float32 dask.array<chunksize=(1, 1664, 1664), meta=np.ndarray>
    group_energy    (time, lat, lon) float32 dask.array<chunksize=(1, 1664, 1664), meta=np.ndarray>
    duration_flash  (time, lat, lon) float32 dask.array<chunksize=(1, 1664, 1664), meta=np.ndarray>
    flash_count     (time) int16 1097 1055 1054 1000 996 ... 1236 1272 1231 1202
    group_count     (time) int16 2460 2367 2381 2296 ... 2499 2676 2542 2594
    event_count     (time) int16 7189 6775 6764 6703 ... 6597 7052 6509 6573
Attributes:
    description:  Center for Weather Forecasting and Climate Studies/INPE
    history:      Create in Thu Mar 17 18:47:58 2022
    source:       GLM Lightning Accumulated (5 minutes) - GOES 16 ['OR_GLM-L2...

In [11]:
%%time
#==================================================================================================================#
#                                              IMPORTA BIBLIOTECAS
#==================================================================================================================#
import os
import pandas as pd
from datetime import datetime
import glob
import xarray as xr
import time
import numpy as np

#==================================================================================================================#
#                                               DEFINE CAMINHO
#==================================================================================================================#
dir_5min = 'output/5min/'
dir_diario = 'output/diario/'

#==================================================================================================================#
#                                            DEFINE O PERÍODO DE PROCESSAMENTO
#==================================================================================================================#
# ano, mes e dia INICIAL do período
anoi = 2018
mesi = 2
diai = 1

# ano, mes e dia FINAL do período
anof = 2018
mesf = 6
diaf = 30

# data desejada
date_in = datetime(anoi, mesi, diai)
date_ini = date_in.strftime('%Y%m%d')

date_en = datetime(anof, mesf, diaf)
date_end = date_en.strftime('%Y%m%d')

# Limites do Brasil
lonmin, lonmax, latmin, latmax = -75.0, -33.0, -35.0, 7.0

#==================================================================================================================#
#                                               LOOP NOS DIAS
#==================================================================================================================#
for file in pd.date_range(date_ini, date_end, freq='d'):

    #---------------------------------------------#
    #             Extrai a data
    #---------------------------------------------#
    ano = file.strftime('%Y')
    mes = file.strftime('%m')
    dia = file.strftime('%d')
       
    print('####################################################################')
    print('                PROCESSANDO ===>>>', f'{ano}-{mes}-{dia}')
    print('####################################################################')

    #---------------------------------------------#
    #           Leitura dos dados
    #---------------------------------------------#
    # caminho do diretório dos arquivos
    dir_files = f'{dir_5min}{ano}/{mes}/' 

    # lista os arquivos do dia
    files = sorted(glob.glob(f'{dir_files}S11635949_{ano}{mes}{dia}*nc')) # S11635949_201901140445.nc
    print('1. Montou a lista dos arquivos=', len(files))

    # verifica se existe arquivos daquele dia
    if len(files) > 0:

        # leitura dos dados diários
        glm_dia = xr.open_mfdataset(files, concat_dim='time', combine='nested', decode_times=False).sel(lon=slice(lonmin,lonmax), lat=slice(latmin, latmax))
        print('2. Leu os arquivos daquele dia')

        #---------------------------------------------#
        #           Salva arquivo Netcdf
        #---------------------------------------------#
        # Tutorial sobre como criar DataSet com a biblioteca Xarray: https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html
        # variáveis
        
        # flashes-GASTA MUIOT TEMPO
        flash_dia = np.array(glm_dia['flash'].sum(dim='time'))

        # fecha o arquivo lido
        glm_dia.close()        
        print('3. Calculou o valor total/média')

        # adiciona a dimensão do tempo
        flash_dia = flash_dia[np.newaxis, :, :]
        print('4. Criou a dimensão do tempo')

        # latitudes, longitudes e tempo
        lats = np.array(glm_dia['lat'])
        lons = np.array(glm_dia['lon'])
        times = pd.date_range(f'{ano}-{mes}-{dia}', periods=1)

        # define encoding para compactação dos dados
        encoding = {'flash': {'zlib': True}}

        # define variáveis
        data_vars = {'flash':( ('time', 'lat', 'lon'), flash_dia, {'units': 'ocorrências/64km2', 'long_name': 'Flashes', '_FillValue': 0} )}

        # define coordenadas
        coords = {'time': times, 'lat': lats, 'lon': lons}

        # define atributos
        attrs = {'description': 'Dados do Sensor GLM abordo do satélite GOES-16 Reamostrado para Escala Diária', 'creation_data': str(datetime.now()), 'author':'Enrique V. Mattos', 'email': 'enrique@unifei.edu.br'}
        print('5. Definou variáveis do DataSet')
        
        # gera Dataset
        glm_dia_netcdf = xr.Dataset(data_vars = data_vars, coords = coords, attrs = attrs)
        print('6. Gerou DataSet')

        # cria a pasta de saída
        os.makedirs(f'{dir_diario}{ano}/{mes}', exist_ok=True)

        # salva arquivo netcdf
        glm_dia_netcdf.to_netcdf(f'{dir_diario}{ano}/{mes}/GOES16_GLM_{ano}{mes}{dia}.nc', encoding=encoding)
        print('7. Salvou arquivo netcdf \n')

######################## 2018-02-01 ########################
1. Montou a lista dos arquivos= 288
2. Leu os arquivos daquele dia
3. Calculou o valor total/média
4. Criou a dimensão do tempo
5. Definou variáveis do DataSet
6. Gerou DataSet
7. Salvou arquivo netcdf 

######################## 2018-02-02 ########################
1. Montou a lista dos arquivos= 288
2. Leu os arquivos daquele dia
3. Calculou o valor total/média
4. Criou a dimensão do tempo
5. Definou variáveis do DataSet
6. Gerou DataSet
7. Salvou arquivo netcdf 

######################## 2018-02-03 ########################
1. Montou a lista dos arquivos= 288
2. Leu os arquivos daquele dia
3. Calculou o valor total/média
4. Criou a dimensão do tempo
5. Definou variáveis do DataSet
6. Gerou DataSet
7. Salvou arquivo netcdf 

######################## 2018-02-04 ########################
1. Montou a lista dos arquivos= 288
2. Leu os arquivos daquele dia
3. Calculou o valor total/média
4. Criou a dimensão do tempo
5. Definou variá

In [7]:
glm_dia

<xarray.Dataset>
Dimensions:         (time: 288, lat: 525, lon: 525)
Coordinates:
  * time            (time) float64 1.768e+07 1.768e+07 ... 1.768e+07 1.768e+07
  * lat             (lat) float32 -34.96 -34.88 -34.8 -34.72 ... 6.8 6.88 6.96
  * lon             (lon) float32 -75.0 -74.92 -74.84 ... -33.24 -33.16 -33.08
Data variables:
    spatial_ref     (time) int32 -2147483647 -2147483647 ... -2147483647
    event           (time, lat, lon) float32 dask.array<chunksize=(1, 525, 525), meta=np.ndarray>
    flash           (time, lat, lon) float32 dask.array<chunksize=(1, 525, 525), meta=np.ndarray>
    group           (time, lat, lon) float32 dask.array<chunksize=(1, 525, 525), meta=np.ndarray>
    event_energy    (time, lat, lon) float32 dask.array<chunksize=(1, 525, 525), meta=np.ndarray>
    flash_energy    (time, lat, lon) float32 dask.array<chunksize=(1, 525, 525), meta=np.ndarray>
    group_energy    (time, lat, lon) float32 dask.array<chunksize=(1, 525, 525), meta=np.ndarray>
    duration_flash  (time, lat, lon) float32 dask.array<chunksize=(1, 525, 525), meta=np.ndarray>
    flash_count     (time) int16 1097 1055 1054 1000 996 ... 1236 1272 1231 1202
    group_count     (time) int16 2460 2367 2381 2296 ... 2499 2676 2542 2594
    event_count     (time) int16 7189 6775 6764 6703 ... 6597 7052 6509 6573
Attributes:
    description:  Center for Weather Forecasting and Climate Studies/INPE
    history:      Create in Thu Mar 17 18:47:58 2022
    source:       GLM Lightning Accumulated (5 minutes) - GOES 16 ['OR_GLM-L2...

# Testes

In [2]:
# arquivos por dia. São arquivos a cada 5min
12*24

288

In [1]:
import xarray as xr
import glob
files = sorted(glob.glob('5min/2019/01/S11635949_20190114*.nc'))
glm_dia = xr.open_mfdataset(files, concat_dim='time', combine='nested', decode_times=False) 
glm_dia

<xarray.Dataset>
Dimensions:         (lat: 3047, lon: 3107, time: 284)
Coordinates:
  * lat             (lat) float32 -66.56 -66.48 -66.4 ... 66.32 66.4 66.48
  * lon             (lon) float32 -141.6 -141.5 -141.4 ... -8.68 -8.6 -8.52
  * time            (time) float64 1.769e+07 1.769e+07 ... 1.769e+07 1.769e+07
Data variables:
    spatial_ref     (time) int32 -2147483647 -2147483647 ... -2147483647
    event           (time, lat, lon) float32 dask.array<chunksize=(1, 3047, 3107), meta=np.ndarray>
    flash           (time, lat, lon) float32 dask.array<chunksize=(1, 3047, 3107), meta=np.ndarray>
    group           (time, lat, lon) float32 dask.array<chunksize=(1, 3047, 3107), meta=np.ndarray>
    event_energy    (time, lat, lon) float32 dask.array<chunksize=(1, 3047, 3107), meta=np.ndarray>
    flash_energy    (time, lat, lon) float32 dask.array<chunksize=(1, 3047, 3107), meta=np.ndarray>
    group_energy    (time, lat, lon) float32 dask.array<chunksize=(1, 3047, 3107), meta=np.ndarray>
    duration_flash  (time, lat, lon) float32 dask.array<chunksize=(1, 3047, 3107), meta=np.ndarray>
    flash_count     (time) float32 1.124e+03 1.108e+03 ... 1.013e+03 982.0
    group_count     (time) float32 2.423e+03 2.458e+03 ... 2.318e+03 2.269e+03
    event_count     (time) float32 6.947e+03 7.034e+03 ... 6.338e+03 6.175e+03
Attributes:
    description:  Center for Weather Forecasting and Climate Studies/INPE
    history:      Create in Fri Mar 18 22:45:59 2022
    source:       GLM Lightning Accumulated (5 minutes) - GOES 16 ['OR_GLM-L2...

In [ ]:
import xarray as xr
ds = xr.open_dataset('output/diario/2024/04/GOES16_GLM_20240401.nc')
ds

In [ ]:
import xarray as xr
ds = xr.open_dataset('output/5min/2024/04/01/S11635949_202404010000.nc')
ds